<a href="https://colab.research.google.com/github/Luicazen/NLPTweets/blob/SVM/initSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.svm import SVC
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as textb
from sklearn import decomposition

In [4]:
from google.colab import files
uploaded = files.upload()
import pandas as pd
import numpy as np

Saving train.csv to train.csv


**Setting up the training data**




In [5]:
#Read in Training data
trainingDF = pd.read_csv('train.csv')
#Setting index to start at 1
trainingDF.index +=1
print("Training Data Summary")
#printing summary features: Shape
display(trainingDF.shape)
#Stats
display(trainingDF.describe())
#First 5 rows
display(trainingDF.head())
#type summary
trainingDF.info()
#column names
print(trainingDF.columns)

Training Data Summary


(7613, 5)

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


,id,keyword,location,text,target
1,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
2,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
3,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
4,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
5,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 1 to 7613
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB
Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')


In [6]:
#Check for null data
print("Training Data")
trainingDF.isnull().sum()

Training Data


,0
id,0
keyword,61
location,2533
text,0
target,0


*As shown above, most NaN data is found in either the keyword or location feild.*

In [7]:
#drop the target column from the training data X
X = trainingDF.drop(['target'], axis=1)
#Y is only the target column
y = trainingDF['target']


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train.shape, X_test.shape

((6090, 4), (1523, 4))

In [21]:
# Initialize TfidfVectorizer to convert text to a matrix of TF-IDF features
#stop_words='english' removes common english words like "the", "is", "and"
#ngram_ranger uses both single words and pairs of words
tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))

# Fit and transform the 'text' column for training and validation sets
xtrain_tfv = tfidf_vec.fit_transform(X_train['text'])
xvalid_tfv = tfidf_vec.transform(X_test['text'])

print(f"Shape of xtrain_tfv: {xtrain_tfv.shape}")
print(f"Shape of xvalid_tfv: {xvalid_tfv.shape}")

Shape of xtrain_tfv: (6090, 61346)
Shape of xvalid_tfv: (1523, 61346)


In [22]:
# support vector decomposition SVD to standardize the data
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

# Scale the data obtained from SVD.
scl = StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

In [18]:
clf = SVC(C=1.0, probability=True)
clf.fit(xtrain_svd_scl, y_train)
predictions = clf.predict_proba(xvalid_svd_scl)

In [20]:
# Convert probabilities to predicted class labels (0 or 1)
y_pred = (predictions[:, 1] > 0.5).astype(int)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.4f}")

F1 Score: 0.7148


Benchmark F1 score for an un-tuned SVM is: 0.7148